In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use all onehot nominal encoding for coupon type: Coffee House

In [2]:
coupon_type = "CoffeeHouse"

In [3]:
encoding_type = 'onehot' ## use this string to access input file and name output files

os.makedirs(os.path.join('model', encoding_type), exist_ok=True)

In [4]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv(f'data/{coupon_type}_data_{encoding_type}.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 49.86% Yes
After SMOTE upsampling
	Num training samples: 3,200 (50.62% yes)
	Num testing samples:  800 (47.50% yes)


## Testing RandomForestClassifier with Grid Search Cross-validation


## RandomForestClassifier Parameters

The main parameters to adjust when using these methods:
* n_estimators
* max_features (sqrt is generally best default for classification. I have 55 features, so try values around 8) 

Make sure to include params:
* max_depth=None
* min_samples_split=2 
* oob_score=True (with bootstrapping)

In [5]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
#                 'param_min_samples_leaf', 
                'param_max_depth', 'param_max_leaf_nodes',
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [6]:
param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3],
            'n_estimators': [100, 150, 200],
#             'min_samples_leaf': [1, 2, 3, 4, 5, 10],
            'max_depth': [None, 20, 25, 30],
            'max_leaf_nodes': [None, 100, 200]
           }

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 20, 25, 30],
                         'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15],
                         'max_leaf_nodes': [None, 100, 200],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_depth=30, max_features=10, min_samples_split=3,
                       n_estimators=200, oob_score=True, random_state=42)

In [7]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.7665625
ROC AUC: TEST 0.864944



In [8]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,param_max_depth,param_max_leaf_nodes,mean_test_score,std_test_score,mean_train_score,std_train_score
689,1,10,3,200,30,None,0.84526391,0.01393578,0.99988945,0.00003141
470,2,9,2,200,25,None,0.84506661,0.01302333,0.99999872,0.00000112
95,3,10,3,200,None,None,0.84506466,0.01307133,0.99989335,0.00003457
473,4,9,3,200,25,None,0.84492792,0.01471283,0.99987138,0.00002428
94,5,10,3,150,None,None,0.84487908,0.01361291,0.99988371,0.00002899
23,6,6,3,200,None,None,0.84470132,0.01456381,0.99983719,0.00002820
761,7,14,3,200,30,None,0.84465346,0.01279128,0.99991802,0.00002717
469,8,9,2,150,25,None,0.84464467,0.01305553,0.99999884,0.00000089
239,9,7,3,200,20,None,0.84455384,0.01436916,0.99956579,0.00007365
149,10,13,3,200,None,None,0.84451086,0.01275523,0.99990654,0.00003048


In [9]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
77,CoffeeHouse_never,0.09675006
20,expiration_2h,0.03045847
7,destination_No Urgent Place,0.02383235
76,CoffeeHouse_less1,0.02191402
92,minsToCouponDest_5-14,0.02155563
21,gender_Male,0.01978961
89,Restaurant20To50_less1,0.01929720
73,Bar_never,0.01847222
74,CoffeeHouse_4~8,0.01836899
18,time_6PM,0.01801773


In [10]:
import pickle

with open(f'model/{encoding_type}/{coupon_type}_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open(f'model/{encoding_type}/{coupon_type}_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [11]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/{coupon_type}_{k}_{encoding_type}.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)

Saving file: data/CoffeeHouse_X_train_onehot.csv
Saving file: data/CoffeeHouse_X_test_onehot.csv
Saving file: data/CoffeeHouse_y_train_onehot.csv
Saving file: data/CoffeeHouse_y_test_onehot.csv


# KNN

In [12]:
param_grid={'n_neighbors': [x for x in range(1,26)]}

clf = KNeighborsClassifier()

grid_search_knn = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search_knn.fit(X_train.values, y_train.values)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25]},
             return_train_score=True, scoring='roc_auc')

In [13]:
knn_cols_to_view = ['rank_test_score', 'param_n_neighbors', 
                    'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

cv_results_knn = pd.DataFrame(grid_search_knn.cv_results_)
cv_results_knn.loc[:, knn_cols_to_view].sort_values('rank_test_score').head(3)

,rank_test_score,param_n_neighbors,mean_test_score,std_test_score,mean_train_score,std_train_score
6,1,7,0.79697414,0.01317671,0.88704088,0.00303974
7,2,8,0.79396390,0.01306790,0.87952866,0.00387925
8,3,9,0.79362498,0.01371128,0.87535931,0.00327425


In [14]:
best_model_knn = grid_search_knn.best_estimator_
best_model_knn
y_test_score_knn = best_model_knn.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_knn):.6f}\n")

KNeighborsClassifier(n_neighbors=7)

ROC AUC: TEST 0.815263



In [15]:
with open(f'model/{encoding_type}/{coupon_type}_GridSearchCV_KNN.pickle', "wb") as f:
    pickle.dump(grid_search_knn,f)
    
with open(f'model/{encoding_type}/{coupon_type}_best_model_KNN.pickle', "wb") as f:
    pickle.dump(best_model_knn,f)

# Logistic regression

In [16]:
## standardization
std_scaler=StandardScaler().fit(X_train) ## first fit the model on the training data

## now std scaler transform
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

## apply regression
liblinear_model = LogisticRegression(solver='liblinear', random_state=42)
liblinear_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(liblinear_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")
    
# use trained model to get predicted target var
y_test_score_ll = liblinear_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_ll)}\n")

## apply regression again, using a different solver
lbfgs_model = LogisticRegression(solver='lbfgs', random_state=42)
lbfgs_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(lbfgs_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")

# use trained model to get predicted target var
y_test_score_lb = lbfgs_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_lb)}\n")

LogisticRegression(random_state=42, solver='liblinear')

5-fold CV ROC AUC: mean: 0.782384 std dev:0.013103
ROC AUC: TEST 0.7757268170426065



LogisticRegression(random_state=42)

5-fold CV ROC AUC: mean: 0.782265 std dev:0.013511
ROC AUC: TEST 0.7757518796992481



In [17]:
## both LR models have essentially the same score, just save one
with open(f'model/{encoding_type}/{coupon_type}_best_model_LogisticRegression.pickle', "wb") as f:
    pickle.dump(liblinear_model,f)